- ## <b>Acompanhamento de precificação: Medicamentos</b>


In [4]:
import pandas as pd
from datetime import datetime
from bmldev.pdleo import cria_dfs, le_base_excel, converte_tipos_colunas, branco_para_nan
from datetime import datetime

In [5]:
dir_input = "./bases/"
dir_output = "./relatorio/"

In [6]:
dt_hoje = '20210609'
dt_ultimo_relatorio = '20210609'

- ## <b>Leitura de bases</b>

In [8]:
%%time
dfs_txt = cria_dfs(
                nomes_bases=['zbfa'],
    
                palavras_chave=['abcfarma'],
    
                cols=[16],
    
                multiplas_bases_txt=[False],
                
                diretorio=dir_input

    
)

Lendo base zbfa...ok
Wall time: 1.9 s


In [50]:
df_dadosfarma = le_base_excel(dir_input, 'banco de dados')

In [38]:
df_iqvia = le_base_excel(dir_input, 'IQVIA_Norte')

In [40]:
## Vamos utilizar para comparar o último desconto sobre PF cadastrado para cada SKU

df_desc_ant =le_base_excel(dir_input, 'desonto', 'Original')

- ## <b>Processamento</b>

- ### df_zbfa

In [18]:
df_zbfa = dfs_txt['zbfa'].copy()

In [20]:
df_zbfa = df_zbfa.iloc[:,:12]

In [21]:
df_zbfa.columns = ['ean', 'pf_18', 'pmc_18', 'pf_17', 'pmc_17', 'pf_17_5', 'pmc_17_5', 'data', 'pf_17_alc', 'pmc_17_alc', 'pf_17_5_alc', 'pmc_17_5_alc']

In [22]:
branco_para_nan(df_zbfa, sobrescreve=True)

In [23]:
zbfa_float_col = [col for col in df_zbfa.columns if [p for p in ['pf', 'pmc'] if p in col]]

In [25]:
df_zbfa[zbfa_float_col] = df_zbfa[zbfa_float_col].astype(float)

In [27]:
converte_tipos_colunas(df_zbfa, {'ean':str, 'data':'data'}, sobrescreve=True)

In [28]:
df_zbfa = df_zbfa.sort_values('data', ascending=False).drop_duplicates('ean').drop(columns='data')

- ### df_desc_ant

In [44]:
df_desc_ant =df_desc_ant[['codigo_de_barras_do_medicamento','material','desc_pf','desc_pmc']].copy()

In [45]:
df_desc_ant.columns = ['ean','material','ult_desc_pf','ult_desc_pmc']

In [47]:
converte_tipos_colunas(df_desc_ant, {'ean':str, 'material':str}, sobrescreve=True)

C:\Users\nathalia.tavares\Documents\Desconto sobre PF Med\bmldev\pdleo.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(tipo)


- ### df_iqvia

In [58]:
## Mantendo tudo o que  temos de acordo com o relatório do IQVIA
df_iqvia = df_iqvia[df_iqvia['status'] !='Não tem']

- ### df_dadosfarma


In [60]:
df_dadosfarma.columnsdd

Index(['material', 'ean', 'categ_loja', 'descricao', 'marca', 'condicao',
       'curva_vlr', 'curva_qtd', 'cod_forn_reg_x', 'nome_forn_reg_x',
       'cod_forn_ult_x', 'nome_forn_ult_x', 'pmm', 'cmv_ult', 'prc_vnd_mao',
       'prc_vnd_pvh', 'prc_vnd_acr', 'prc_vnd_bva', 'analista',
       'cod_forn_reg_y', 'nome_forn_reg_y', 'cod_forn_ult_y',
       'nome_forn_ult_y', 'margem', 'pf_18', 'pmc_18', 'pf_17', 'pmc_17',
       'pf_17_5', 'pmc_17_5', 'pf_17_alc', 'pmc_17_alc', 'pf_17_5_alc',
       'pmc_17_5_alc'],
      dtype='object')

In [63]:
df_dadosfarma[['material','categ_loja','condicao','cod_forn_ult_x', 'nome_forn_ult_x', 'pmm', 'cmv_ult']]

,material,categ_loja,condicao,cod_forn_ult_x,nome_forn_ult_x,pmm,cmv_ult
0,4000010,NaN,ND,4916.0,MAPEMI-BRASIL MATERIAIS MÉDICOS E O,9.32,9.38
1,4000011,NaN,ND,3593.0,SB COMERCIO LTDA,6.49,6.49
2,4000012,NaN,ND,12585.0,TOTAL LOGÍSTICA FARMACÊUTICA LTDA -,11.64,11.64
3,4000013,SIMILAR,ND,12999.0,M DOS S LOFIEGO EIRELI,3.27,5.30
4,4000014,SIMILAR,RP,12585.0,TOTAL LOGÍSTICA FARMACÊUTICA LTDA -,2.94,2.94
...,...,...,...,...,...,...,...
7845,4012573,REFERENCIA,RP,13001.0,DISTRIBUIDORA DE MEDICAMENTOS SANTA,21.89,16.40
7846,4012584,REFERENCIA,RP,13001.0,DISTRIBUIDORA DE MEDICAMENTOS SANTA,36.03,28.91
7847,4012586,REFERENCIA,RP,13001.0,DISTRIBUIDORA DE MEDICAMENTOS SANTA,19.49,15.64
7848,4012587,NaN,RP,13001.0,DISTRIBUIDORA DE MEDICAMENTOS SANTA,56.33,65.51
